In [ ]:
# GPU tagging time for ~60000 posts: ~30 minutes

# mount drive to access data
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# change path HERE to reflect your folder structure
my_path = '/content/drive/MyDrive/suicide-project/data/'

In [ ]:
!head -2 /content/drive/MyDrive/suicide-project/data/SN-only-FILTERED_PROCESSED_TEXT_CORPUS_v2.csv

post_id,lemmatized_body,keywords_matched,group_name,year,month
241976,nervous planning reading lot sn dmso get wednesday im afraid fucking dying horrible death sn sound like really awful way go screw,['sn'],sn,2019,2


In [ ]:
# create dataframe from csv data file

import pandas as pd
from ast import literal_eval
df = pd.read_csv(my_path + 'SN-only-FILTERED_PROCESSED_TEXT_CORPUS_v2.csv', usecols = ['post_id','lemmatized_body','year','month'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62178 entries, 0 to 62177
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   post_id          62178 non-null  int64 
 1   lemmatized_body  62178 non-null  object
 2   year             62178 non-null  int64 
 3   month            62178 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 1.9+ MB


In [ ]:
# flair product ner
!pip install flair

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.1/373.1 KB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 KB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 KB 8.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 KB 37.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.5/788.5 KB 55.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 78.4 MB/s eta 0:00:0

In [ ]:
from flair.data import Sentence
from flair.models import SequenceTagger

tagger = SequenceTagger.load("flair/ner-english-ontonotes-large")

2023-04-10 07:11:02,610 SequenceTagger predicts: Dictionary with 76 tags: <unk>, O, B-CARDINAL, E-CARDINAL, S-PERSON, S-CARDINAL, S-PRODUCT, B-PRODUCT, I-PRODUCT, E-PRODUCT, B-WORK_OF_ART, I-WORK_OF_ART, E-WORK_OF_ART, B-PERSON, E-PERSON, S-GPE, B-DATE, I-DATE, E-DATE, S-ORDINAL, S-LANGUAGE, I-PERSON, S-EVENT, S-DATE, B-QUANTITY, E-QUANTITY, S-TIME, B-TIME, I-TIME, E-TIME, B-GPE, E-GPE, S-ORG, I-GPE, S-NORP, B-FAC, I-FAC, E-FAC, B-NORP, E-NORP, S-PERCENT, B-ORG, E-ORG, B-LANGUAGE, E-LANGUAGE, I-CARDINAL, I-ORG, S-WORK_OF_ART, I-QUANTITY, B-MONEY


In [ ]:
posts = df['lemmatized_body'].tolist()

In [ ]:
postidx = []
products = []
for idx, post in enumerate(posts):
  sentence = Sentence(post)
  tagger.predict(sentence)
  for entity in sentence.get_spans('ner'):
    ent_label = entity.get_label()
    if ent_label.value == 'PRODUCT':
      postidx.append(idx)
      products.append(ent_label.data_point.text)

In [ ]:
upostidx = list(set(postidx))
print(len(postidx), len(upostidx))

1690 1157


In [ ]:
df2 = df.iloc[upostidx]
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1157 entries, 18433 to 4095
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   post_id          1157 non-null   int64 
 1   lemmatized_body  1157 non-null   object
 2   year             1157 non-null   int64 
 3   month            1157 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 45.2+ KB


In [ ]:
from collections import defaultdict
products_dict = defaultdict(list)

for postix, prod in zip(postidx,products):
  products_dict[postix].append(prod)

In [ ]:
len(products_dict)

1157

In [ ]:
products_s = pd.Series(products_dict)
products_s.info()

<class 'pandas.core.series.Series'>
Int64Index: 1157 entries, 85 to 62100
Series name: None
Non-Null Count  Dtype 
--------------  ----- 
1157 non-null   object
dtypes: object(1)
memory usage: 18.1+ KB


In [ ]:
df3 = df2.join(products_s.to_frame())
df3.rename(columns={0: 'products'}, inplace=True)
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1157 entries, 18433 to 4095
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   post_id          1157 non-null   int64 
 1   lemmatized_body  1157 non-null   object
 2   year             1157 non-null   int64 
 3   month            1157 non-null   int64 
 4   products         1157 non-null   object
dtypes: int64(3), object(2)
memory usage: 86.5+ KB


In [ ]:
df3

,post_id,lemmatized_body,year,month,products
18433,834642,rule sodium nitrite nembutal popular reliable ...,2020,8,"[nembutal, nembutal, nembutal]"
20482,1336164,went drug store got everything needed sn quest...,2021,9,[nexium]
8195,779358,cool jet dude reason humour cant used suicide ...,2020,7,[50100ml]
51203,1629078,id say antacid tossup right pph recently dropp...,2022,6,[dramamine]
38920,531160,thanks help worried 100 tab 90mg bupropian 70 ...,2020,1,[bupropian]
...,...,...,...,...,...
61425,813350,’ consider three major attempt one combination...,2020,8,[pregabalin]
59384,412122,donγçöt think mementomori said try put sn caps...,2019,9,"[domperidone, domperidone]"
61436,749985,laura123 sorry birthday last month turned 23 b...,2020,6,"[firefox, firefox]"
40958,92864,read common side effect raglan two concern ins...,2018,10,[raglan]


In [ ]:
df3.to_csv(my_path+'product-ner.csv',index=False)

In [ ]:
productss = sorted(list(set(products)))
productss
with open(my_path+'txt/products-sorted.txt','w') as f:
  for p in productss:
    f.write(p)
    f.write('\n')